In [19]:
import os
import sys
import random
import math
import numpy as np
import tensorflow as tf
import skimage.io
import matplotlib
matplotlib.use('Agg')
import cv2

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

C:\Users\qianhuil\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\Users\qianhuil\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\qianhuil\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\qianhuil\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\qianhuil\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\qianhuil\Anaconda3\envs\tensorfl

In [13]:
# Root directory of the project
ROOT_DIR = os.path.abspath("D:\Summer\crowdsourcing\Mask_RCNN-master\Mask_RCNN-master")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)
#image_dir
IMAGE_DIR = os.path.join(ROOT_DIR, "images")
IMAGE_DIR_2 = os.path.join(ROOT_DIR, "images_blur")

Using TensorFlow backend.
C:\Users\qianhuil\Anaconda3\envs\tensorflow\lib\site-packages\pycocotools\coco.py:49: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\Users\qianhuil\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\qianhuil\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\qianhuil\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\qianhuil\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\qianhui

In [14]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'mrcnn_mask_loss': 1.0, 'mrcnn_class_loss': 1.0, 'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_bbox_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

In [15]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [17]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [ ]:
def segmentation(image_dir):
    image = skimage.io.imread(image_dir)
    # Create model object in inference mode.
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

    # Load weights trained on MS-COCO
    model.load_weights(COCO_MODEL_PATH, by_name=True)
    

    results = model.detect([image], verbose=1)

    # Visualize results
    r = results[0]
    # person_list = [i for i, e in enumerate(r['class_ids']) if e ==1]    
    test_image = visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                class_names, r['scores'])    
    #get person_class pixel out of r['masks']
    person_list = [i for i, e in enumerate(r['class_ids']) if e ==1]    
    pix_list = []
    if len(person_list) != 0:
        for i in person_list:
            for row in range(r['rois'][i][0],r['rois'][i][2]+1):
                for position in range(len(r['masks'][0])):
                    if r['masks'][row][position][i] == True:
                        pix_list.append([row,position])
    return(pix_list)

In [99]:
def contour(pix_list,black_dir):
    img = cv2.imread(black_dir)# black image with same size
    print(len(img[0]))
    new = img
    for item in pix_list:
    new[item[0]][item[1]] = [255,255,255]
    new = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(new,127,255,0)
    im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    return contours
    # print(len(contours))

In [103]:
def BlurContours(image, contours, ksize, sigmaX, *args):
    sigmaY = args[0] if len(args) > 0 else sigmaX
    mask = np.zeros(image.shape[:2])
    for i, contour in enumerate(contours):
        cv2.drawContours(mask, contour, i, 255, -1)
    blurred_image = cv2.GaussianBlur(image, (ksize, ksize), sigmaX, None, sigmaY)
    result = np.copy(image)
    alpha = mask/255.
    result = alpha[:, :, None]*blurred_image + (1-alpha)[:, :, None]*result
    return result


In [ ]:
if __name__ == "__main__":     
    black_dir = 'D:\Summer\crowdsourcing\Mask_RCNN-master\Mask_RCNN-master\black.png'
    foldername= os.listdir(IMAGE_DIR)
    j = 0
    for subfolder in foldername: #subfolder,bottom_0\
        path = os.path.join(ROOT_DIR, subfolder)
        for image_name in os.listdir(path): #image_name,01fb7befe10e876edb89e2935215f5603dd8f12d2b94363a37eae713.png
            image_path = os.path.join(path,image_name)
            image_set_list = segmentation(image_path)
            image_con = contour(image_seg_list,black_dir)
            image = cv2.imread(image_path)
            for i in range(len(contours)):
                contour_list = []
                for j in contours[i]:
                    contour_list.append(j)
                
                blur_image=BlurContours(image,[[np.array(contour_list)]],17,5)
#                 cv2.drawContours(image, contours, -1, (0,255,0), 3)
            new_image = blur_image[..., ::-1]
            # print(new_image)
            new_path = os.path.join(IMAGE_DIR_2,subfolder,image_name)
            cv2.imwrite(new_path,new_image)
            j = j+1
            if j % 100 == 0:
                    print("100 done",j)
    

In [3]:
#for test code
def list_files(dir):
    r = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            r.append(os.path.join(root, name))
    return r
a = list_files('D:\Summer\crowdsourcing\Mask_RCNN-master\Mask_RCNN-master\images')
print(a[0])

D:\Summer\crowdsourcing\Mask_RCNN-master\Mask_RCNN-master\images\bottom_0\01fb7befe10e876edb89e2935215f5603dd8f12d2b94363a37eae713.png


In [18]:
#for test code
IMAGE_DIR = 'D:\Summer\crowdsourcing\Mask_RCNN-master\Mask_RCNN-master\images'
foldername= os.listdir (IMAGE_DIR)
image_list = []
for subfolder in foldername:
    path = os.path.join(IMAGE_DIR, subfolder)
    for image in os.listdir(path):
        image_path = os.path.join(path,image)
        image_list.append(image_path)
print(image_list[0])

D:\Summer\crowdsourcing\Mask_RCNN-master\Mask_RCNN-master\images\bottom_0\01fb7befe10e876edb89e2935215f5603dd8f12d2b94363a37eae713.png


In [ ]:
#for test code
for i in range(len(contours)):
    contour_list = []
    for j in contours[i]:
        contour_list.append(j)
    image=BlurContours(image,[[np.array(contour_list)]],17,5)
    cv2.drawContours(image, contours, -1, (0,255,0), 3)
new_image = image[..., ::-1]
# print(new_image)
cv2.imwrite('C:/Users/qianhuil/Desktop/2.jpg',new_image)